In [ ]:
from pathlib import Path

fp = "/home/i53/student/yagmurlu/code/manten/data/maniskill2/demos/%s/motionplanning/trajectory.%s.%s.cpu.%s"


def get_maniskill_filepath(task, obs, action, ext):
    return fp % (task, obs, action, ext)


def load_json(filepath):
    import json

    with Path(filepath).open("r") as f:
        return json.load(f)

In [ ]:
import h5py

data = h5py.File(
    get_maniskill_filepath("PegInsertionSide-v1", "pointcloud", "pd_ee_delta_pose", "h5"), "r"
)
json_data = load_json(
    get_maniskill_filepath("PegInsertionSide-v1", "pointcloud", "pd_ee_delta_pose", "json")
)

In [ ]:
num_episodes = len(json_data["episodes"])

In [ ]:
def inflate_h5_data(data):
    out = {}
    for k in data:
        if isinstance(data[k], h5py.Dataset):
            out[k] = data[k][:]
        else:
            out[k] = inflate_h5_data(data[k])
    return out


def load_episode(data, episode_idx):
    return inflate_h5_data(data[f"traj_{episode_idx}"])

In [ ]:
import numpy as np


def pack_episode_into_trajectories(episode_idx, traj_len=20):
    episode = load_episode(data, episode_idx)
    episode["trajectories"] = []

    for i in range(len(episode["actions"])):
        chunk = episode["actions"][i + 1 : i + 1 + traj_len]
        if len(chunk) < traj_len:
            # pad with last zeroes # assuming delta
            chunk = np.concatenate([chunk, np.zeros((traj_len - len(chunk), chunk.shape[1]))])
        episode["trajectories"].append(chunk)

    episode["trajectories"] = np.array(episode["trajectories"])
    del episode["actions"]
    return episode